In [ ]:
import pandas as pd
import requests
import time

api_key = 'RGAPI-b56cf1c8-b68f-404e-bae4-d0b432ac53b1'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

for rank_level in ['I','II', 'III'] :
    final_reulst = []
    i = 1
    for i in range(1, 10):
        print(rank_level, i)
        url = f"https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GOLD/{rank_level}?page={i}"
        gold_user = requests.get(url, headers=request_header).json()
        final_reulst += gold_user
        time.sleep(1)
        if len(gold_user) == 0:
            break
    df= pd.json_normalize(final_reulst)
    df.to_csv(f'gold_user.csv', index=False, encoding='CP949')

In [ ]:
import requests
import pprint
import pandas as pd
import datetime
import csv
import time

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-b56cf1c8-b68f-404e-bae4-d0b432ac53b1'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

def getPuuid(encryptedSummonerId):
    url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{encryptedSummonerId}"
    return requests.get(url, headers=request_header).json()['puuid']

def match_v5_get_list_match_id(puuid, start, count):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
    return requests.get(url, headers=request_header).json()

def get_match_id(puuid) :
    f = open('MatchId_gold_data.csv', 'a', newline='') #
    wr = csv.writer(f)
    for k in [0,100,200] :
        time.sleep(2)
        match_data = match_v5_get_list_match_id(puuid, k, 100)
        for i in match_data :
            wr.writerow([i])


summonerEncryptedId = pd.read_csv('C:/Users/dlehr/Desktop/riotData/gold_user.csv',encoding='cp949') #
# print(league_df['summonerName'])
id=summonerEncryptedId['summonerId']
i = 1
error_list = []
for enc_id in id[:10] :
    try:
        print(enc_id, i)
        puuid = getPuuid(enc_id)
        time.sleep(1)
        get_match_id(puuid)
        # if puuid.status_code==404 :
        #     raise ValueError
#         final_reulst.append(puuid)
#         time.sleep(1)
    except :
        print('제외됨')
        error_list.append(i)

    i += 1

print(error_list)

In [ ]:
import pandas as pd
import pprint
import requests
import time
import csv

pp = pprint.PrettyPrinter(indent=4)
api_key = 'RGAPI-b56cf1c8-b68f-404e-bae4-d0b432ac53b1'
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

detail_key = ['assists', 'championId', 'deaths', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist',
              'firstTowerKill', 'individualPosition', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6',
              'kills', 'longestTimeSpentLiving', 'participantId', 'perks', 'summoner2Id', 'summonerName', 'teamId',
              'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal', 'totalMinionsKilled',
              'totalTimeSpentDead', 'visionScore']


def get_data_from_dict(dict_key):
    rst = {}
    for value in dict_key:
        for key in detail_key:
            if value == key:
                if key == 'perks':
                    dict_key[key]['statPerks'] = [dict_key[key]['statPerks']['defense'],
                                                  dict_key[key]['statPerks']['flex'],
                                                  dict_key[key]['statPerks']['offense']]
                    primary = dict_key[key]['styles'][0]
                    sub = dict_key[key]['styles'][1]
                    dict_key[key]['styles_primary'] = [primary['selections'][0]['perk'],
                                                       primary['selections'][1]['perk'],
                                                       primary['selections'][2]['perk'],
                                                       primary['selections'][3]['perk']]
                    dict_key[key]['styles_sub'] = [sub['selections'][0]['perk'], sub['selections'][1]['perk']]
                    del dict_key['perks']['styles']
                rst[value] = dict_key[key]
    return rst


def get_objective_data(ob_data):
    result = {}
    for k in ob_data:
        if ob_data[k]['first'] == True:
            ob_data[k]['first'] = 1
        else:
            ob_data[k]['first'] = 0
        result[k] = [ob_data[k]['first'], ob_data[k]['kills']]
    return result


def main(match_id):
    url = f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}"
    meta_data = requests.get(url, headers=request_header).json()
    
    final_result = {}
    final_result['match_id'] = match_id
    final_result['Duration'] = meta_data['info']['gameDuration']
    final_result['Version'] = meta_data['info']['gameVersion']
    participants = meta_data['info']['participants']

    for player in range(len(participants)):
        player_data = get_data_from_dict(participants[player])
        final_result[f'player_{player}'] = player_data

    ban_list_blue_team, ban_list_red_team = [], []
    for blue in meta_data['info']['teams'][0]['bans']:
        ban_list_blue_team.append(blue['championId'])

    for red in meta_data['info']['teams'][1]['bans']:
        ban_list_red_team.append(red['championId'])

    final_result['blue_ban_list'] = ban_list_blue_team
    final_result['red_ban_list'] = ban_list_red_team
    final_result['blue_object_list'] = get_objective_data(meta_data['info']['teams'][0]['objectives'])
    final_result['red_object_list'] = get_objective_data(meta_data['info']['teams'][1]['objectives'])

    if meta_data['info']['teams'][0]['win'] == False:
        win_team = meta_data['info']['teams'][1]['teamId']
    else:
        win_team = meta_data['info']['teams'][0]['teamId']
    final_result['win_team'] = win_team
    return final_result


with open('C:/Users/dlehr/Desktop/riotData/MatchId_gold_data.csv','r')as f:
    df = [line.strip() for line in f]
read_data = df
result_array = []

for i in range(0,len(read_data)):
    try:
        final_result = main(read_data[i])
        result_array.append(final_result)
        print(f"{i}번셀 완료, ", f'k={i} 완료, ', read_data[i])
        i += 1
        if i == len(read_data)-1:
            keys = list(final_result.keys())
            result = pd.json_normalize(result_array)
            result.to_csv(f'gold_ingame.csv', index=False, mode='a', encoding='CP949')
    except Exception as e:
        print('error', f'k={i} 완료')
        print(e)
        result = pd.json_normalize(result_array)
        if i == len(read_data)-1:
            keys = list(final_result.keys())
            result = pd.json_normalize(result_array)
            result.to_csv(f'gold_ingame.csv', index=False, mode='a', encoding='CP949')
        continue